à partir de tous les datasets dispos (5 dernières années), on veut nettoyer et rassembler les données.

Voici la liste des datasets à récupérer :
* https://files.data.gouv.fr/geo-dvf/latest/csv/2017/full.csv.gz
* https://files.data.gouv.fr/geo-dvf/latest/csv/2018/full.csv.gz
* https://files.data.gouv.fr/geo-dvf/latest/csv/2019/full.csv.gz
* https://files.data.gouv.fr/geo-dvf/latest/csv/2020/full.csv.gz
* https://files.data.gouv.fr/geo-dvf/latest/csv/2021/full.csv.gz


## Téléchargement des datasets

In [1]:
from pathlib import Path
import requests
from tqdm import tqdm

BASE_URL = "https://files.data.gouv.fr/geo-dvf/latest/csv"
BASE_FILENAME = "full"
EXTENSION = ".csv.gz"

# url dataset example : 
# https://files.data.gouv.fr/geo-dvf/latest/csv/2017/full.csv.gz
def create_url(year: int) -> str:
    return f"{BASE_URL}/{year}/{BASE_FILENAME}{EXTENSION}"

YEARS = range(2017, 2022)
DATASET_FOLDER_PATH = Path("./dvf")
DATASET_FOLDER_PATH.mkdir(parents=True, exist_ok=True)

def download_file(url: str, destination: Path):
    response = requests.get(url)
    with destination.open("wb") as file:
        file.write(response.content)

In [2]:
for year in tqdm(YEARS, desc="Download csv files"):
    url = create_url(year)
    output_file_path = DATASET_FOLDER_PATH / f"{year}{EXTENSION}"
    download_file(url, output_file_path)
    
print()
print(f"CSV files : {[p.as_posix() for p in DATASET_FOLDER_PATH.rglob('*')]}")

Download csv files: 100%|██████████| 5/5 [00:20<00:00,  4.01s/it]


CSV files : ['dvf/2021.csv.gz', 'dvf/2018.csv.gz', 'dvf/2020.csv.gz', 'dvf/2019.csv.gz', 'dvf/2017.csv.gz']


## Chargement et fusion des datasets en un dataset unique

Le chargement des datasets en mémoire directement n'est pas possible (trop de RAM nécéssaire)

Il faut donc nettoyer fichier par fichier avant de fusionner les datasets

In [5]:
import pandas as pd
import numpy as np

def load_dataset(path: Path) -> pd.DataFrame:
    print(f"==== Load dataset from csv {path.as_posix()} ====")
    return pd.read_csv(path, low_memory=False)

def remove_type_locals(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(df[(
        (df["type_local"] == "Dépendance")
        | (df["type_local"] == "Local industriel. commercial ou assimilé")
    )].index)

def remove_useless_columns(df: pd.DataFrame) -> pd.DataFrame:
    useless_columns = [
        "ancien_code_commune",
        "ancien_nom_commune",
        "ancien_id_parcelle",
        "numero_volume",
        "lot1_numero",
        "lot1_surface_carrez",
        "lot2_numero",
        "lot2_surface_carrez",
        "lot3_numero",
        "lot3_surface_carrez",
        "lot4_numero",
        "lot4_surface_carrez",
        "lot5_numero",
        "lot5_surface_carrez",
        "code_nature_culture",
        "nature_culture",
        "code_nature_culture_speciale",
        "nature_culture_speciale"
    ]
    print("Remove useless columns")
    return df.drop(useless_columns, axis="columns")

def remove_rows_without_required_data(df: pd.DataFrame) -> pd.DataFrame:
    necessary_columns = [
        "date_mutation",
        "nature_mutation",
        "valeur_fonciere",
        "type_local",
        "surface_reelle_bati",
        "nombre_pieces_principales"
    ]
    
    def are_necessary_columns_all_present(row: pd.Series) -> bool:
        return all(pd.notna(row[nc]) for nc in necessary_columns)
    
    print("Remove rows without required data")
    return df.drop(
        df[~df.apply(are_necessary_columns_all_present, axis=1)].index
    )

def remove_expensive_mutations(df: pd.DataFrame) -> pd.DataFrame:
    print("Remove expensive mutations")
    return df[df["valeur_fonciere"] <= 10_000_000]

def remove_cheap_mutations(df: pd.DataFrame) -> pd.DataFrame:
    print("Remove cheap mutations")
    return df[df["valeur_fonciere"] >= 10_000]

def remove_duplicate_id_mutation(df: pd.DataFrame) -> pd.DataFrame:
    print("Remove duplicate mutation ids")
    columns = df.columns.tolist()
    gen = (group.iloc[-1] 
           for _, group in tqdm(df.groupby("id_mutation"), 
                                desc="Remove group duplicates")
    )
    return pd.DataFrame(gen, columns=columns)

def arrange_area_values(df: pd.DataFrame) -> pd.DataFrame:
    print("Arrange area values")
    df.loc[df["surface_terrain"].isna(), "surface_terrain"] = 0.0
    return df

def add_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    print("Add date columns")
    df[["date_mutation_annee", "date_mutation_mois", "date_mutation_jour"]] = (
        df["date_mutation"].str.split("-", 0, expand=True)
    )
    return df

def cast_columns(df: pd.DataFrame) -> pd.DataFrame:
    print("Cast columns")
    df["valeur_fonciere"] = df["valeur_fonciere"].astype(int)
    # Not all rows have a postal code, but they do have a city code (INSEE)
    df["code_postal"] = df["code_postal"].apply(lambda c: f"{int(c):05d}" if pd.notna(c) else "00000")
    df["code_type_local"] = df["code_type_local"].astype(int)
    df["nombre_pieces_principales"] = df["nombre_pieces_principales"].astype(int)
    return df

def save_dataframe(df: pd.DataFrame, source_path: Path, dest_folder: Path):
    basename = source_path.stem.split('.')[0]
    dest_file = dest_folder / f"{basename}_cleaned.csv"

    print(f"Save dataframe to {dest_file.as_posix()}")
    df.to_csv(dest_file)

In [6]:
# Composition time !
def clean_dataframe(df: pd.DataFrame, source_path: Path, dest_folder: Path) -> pd.DataFrame:
    # "pipe" method allows to chain functions on entire dataframe
    # (where "apply" expect functions used for each row / column)
    print(f"==== Clean dataframe from source {source_path.as_posix()} ====")
    df = (df.pipe(remove_type_locals)
        .pipe(remove_useless_columns)
        .pipe(remove_rows_without_required_data)
        .pipe(remove_expensive_mutations)
        .pipe(remove_cheap_mutations)
        .pipe(remove_duplicate_id_mutation)
        .pipe(arrange_area_values)
        .pipe(add_date_columns)
        .pipe(cast_columns)
    )
    if source_path is not None and dest_folder is not None:
        save_dataframe(df, source_path, dest_folder)
    return df

In [9]:
CLEANED_FOLDER_PATH = Path("./dvf_cleaned")
CLEANED_FOLDER_PATH.mkdir(parents=True, exist_ok=True)

datasets_with_path = ((load_dataset(p), p) for p in DATASET_FOLDER_PATH.rglob('*'))
cleaned_datasets = (clean_dataframe(df, p, CLEANED_FOLDER_PATH) 
                    for df, p in datasets_with_path
)

dataset = pd.concat(cleaned_datasets, axis=0, ignore_index=True)
dataset.sort_values(by=['date_mutation'], inplace=True)
dataset

==== Load dataset from csv dvf/2021.csv.gz ====



Remove group duplicates:  51%|█████     | 433451/856291 [01:49<01:36, 4373.23it/s]

==== Clean dataframe from source dvf/2021.csv.gz ====
Remove useless columns
Remove rows without required data
Remove expensive mutations
Remove cheap mutations
Remove duplicate mutation ids


Remove group duplicates: 100%|██████████| 856291/856291 [02:43<00:00, 5222.11it/s]


Arrange area values
Add date columns
Cast columns
Save dataframe to dvf_cleaned/2021_cleaned.csv
==== Load dataset from csv dvf/2018.csv.gz ====
==== Clean dataframe from source dvf/2018.csv.gz ====
Remove useless columns
Remove rows without required data
Remove expensive mutations
Remove cheap mutations
Remove duplicate mutation ids


...

Remove group duplicates: 100%|██████████| 1037654/1037654 [03:19<00:00, 5200.60it/s]


Arrange area values
Add date columns
Cast columns
Save dataframe to dvf_cleaned/2019_cleaned.csv
==== Load dataset from csv dvf/2017.csv.gz ====
==== Clean dataframe from source dvf/2017.csv.gz ====
Remove useless columns
Remove rows without required data
Remove expensive mutations
Remove cheap mutations
Remove duplicate mutation ids


Remove group duplicates: 100%|██████████| 962707/962707 [03:16<00:00, 4899.86it/s]


Arrange area values
Add date columns
Cast columns
Save dataframe to dvf_cleaned/2017_cleaned.csv


,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,date_mutation_annee,date_mutation_mois,date_mutation_jour
3846451,2017-1016305,2017-01-01,1,Vente,149200,124.0,NaN,AV DE GENEVE - ANNECY,1090,74000,...,2,Appartement,56.0,2,0.0,6.124644,45.923373,2017,01,01
4374565,2017-414602,2017-01-01,1,Vente,112000,40.0,NaN,RUE FONTENELLE,0409,33140,...,2,Appartement,47.0,2,0.0,-0.576723,44.771351,2017,01,01
4758043,2017-944970,2017-01-01,1,Vente,140000,17.0,NaN,RUE PAUL VERLAINE,3100,69100,...,2,Appartement,42.0,2,0.0,4.880406,45.768108,2017,01,01
4086169,2017-1331938,2017-01-01,1,Vente,270000,7.0,NaN,RUE EMILE GOEURY,0970,93220,...,1,Maison,94.0,4,451.0,2.544544,48.879254,2017,01,01
4526338,2017-625328,2017-01-02,1,Vente,204750,2.0,NaN,VEN DU SERPENTIN,0603,44119,...,1,Maison,90.0,4,0.0,-1.629645,47.337118,2017,01,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633959,2021-704333,2021-12-31,2,Vente,587703,5.0,NaN,RUE WATTEAU RGE LEFEBVRE,2881,59150,...,1,Maison,50.0,3,25.0,NaN,NaN,2021,12,31
830121,2021-964658,2021-12-31,1,Vente,335000,16.0,NaN,AV DU GRAND CANAL,0394,78450,...,2,Appartement,60.0,3,0.0,1.992803,48.834463,2021,12,31
830161,2021-964706,2021-12-31,1,Vente,5228800,3.0,NaN,RTE DE GRANDCHAMP,0090,78113,...,1,Maison,527.0,12,11300.0,1.617997,48.705250,2021,12,31
445017,2021-442506,2021-12-31,1,Vente,258000,17.0,NaN,ALL PIERRE SIMON DE LAPLACE,0869,37400,...,1,Maison,84.0,4,270.0,0.972299,47.404027,2021,12,31


In [10]:
MERGED_DATASET_PATH = Path("./dvf_merged.csv.gz")
dataset.to_csv(MERGED_DATASET_PATH, compression="gzip")

In [11]:
!cd dvf_cleaned
!gzip dvf_cleaned/2017_cleaned.csv \
    dvf_cleaned/2018_cleaned.csv \
    dvf_cleaned/2019_cleaned.csv \
    dvf_cleaned/2020_cleaned.csv \
    dvf_cleaned/2021_cleaned.csv -v

dvf_cleaned/2017_cleaned.csv:	 74.1% -- replaced with dvf_cleaned/2017_cleaned.csv.gz
dvf_cleaned/2018_cleaned.csv:	 74.0% -- replaced with dvf_cleaned/2018_cleaned.csv.gz
dvf_cleaned/2019_cleaned.csv:	 73.7% -- replaced with dvf_cleaned/2019_cleaned.csv.gz
dvf_cleaned/2020_cleaned.csv:	 73.2% -- replaced with dvf_cleaned/2020_cleaned.csv.gz
dvf_cleaned/2021_cleaned.csv:	 72.1% -- replaced with dvf_cleaned/2021_cleaned.csv.gz


In [12]:
# Download all files
!zip -r dvf_cleaned.zip dvf_cleaned dvf_merged.csv.gz
from google.colab import files
files.download("dvf_cleaned.zip")

  adding: dvf_cleaned/ (stored 0%)
  adding: dvf_cleaned/2019_cleaned.csv.gz (deflated 0%)
  adding: dvf_cleaned/2021_cleaned.csv.gz (deflated 0%)
  adding: dvf_cleaned/2018_cleaned.csv.gz (deflated 0%)
  adding: dvf_cleaned/2017_cleaned.csv.gz (deflated 0%)
  adding: dvf_cleaned/2020_cleaned.csv.gz (deflated 0%)
  adding: dvf_merged.csv.gz (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>